In [1]:
%matplotlib inline
import mobilenet_lib
import os
os.environ['CUDA_VISIBLE_DEVICES']='7'
import tensorflow as tf

from matplotlib import pyplot
import numpy as np

import shutil
from caffe2.proto import caffe2_pb2

from caffe2.python import core, model_helper, workspace, brew, net_drawer, visualize, utils
import caffe2.python.predictor.mobile_exporter as me
core.GlobalInit(["caffe2", "--caffe2-log-level=1"])

In [2]:
class MobileNetBuilder:
    '''
    build an modelnet model
    '''
    def __init__(self, model, width_mult=1):
        self.model = model
        self.previousBlob = ""
        self.depthWiseCnt = 1
        self.pointWiseCnt = 1
        self.width_mult=width_mult
        
        
    def addInputDataAndStandConv(self, data):
        '''
        add data to network & add the first standard convolution layer &
        add the batchnorm & add relu
        '''
        brew.conv(self.model, 
                  data, 
                  "conv1", 
                  dim_in=3, 
                  dim_out=int(32 * self.width_mult), 
                  kernel=3, 
                  stride=2, 
                  pad_r=1, 
                  pad_b=1, 
                  pad_l=0, 
                  pad_t=0,
                  no_bias=True)
        brew.spatial_bn(self.model, 
                        "conv1", 
                        "conv1_spatbn", 
                        int(32 * self.width_mult), 
                        epsilon=1e-3, 
                        is_test=True, 
                        no_bias=False)
        brew.relu(self.model, "conv1_spatbn", "conv1_spatbn_relu")
        self.previousBlob = "conv1_spatbn_relu"
    
    def addDepthwiseConvAndPointWiseConv(self, filter_in, filter_out, isDownSample):
        _dim_in = int(filter_in * self.width_mult)
        _dim_out = int(filter_out * self.width_mult)
        
        # add depthwise layer
        brew.group_conv(self.model,
                        self.previousBlob, 
                        "depthwise%d" % (self.depthWiseCnt),
                        dim_in=_dim_in,
                        dim_out=_dim_in,
                        kernel=3,
                        stride=(1 if isDownSample is False else 2),
                        pad_t = (1 if isDownSample is False else 0), 
                        pad_r = (1 if isDownSample is False else 1), 
                        pad_b = (1 if isDownSample is False else 1), 
                        pad_l = (1 if isDownSample is False else 0),
                        group=_dim_in,
                        no_bias=True
                       )
        # add bn
        brew.spatial_bn(self.model, 
                        "depthwise%d" % (self.depthWiseCnt),
                        "depthwise%d_spatbn" % (self.depthWiseCnt), 
                        _dim_in, 
                        epsilon=1e-3, 
                        is_test=True,
                        no_bias=False
                       )
        # add relu
        brew.relu(self.model, 
                  "depthwise%d_spatbn" % (self.depthWiseCnt), 
                  "depthwise%d_relu" % (self.depthWiseCnt))
        
        # add conv
        brew.conv(self.model, 
                  "depthwise%d_relu" % (self.depthWiseCnt), 
                  "pointwise%d" % (self.pointWiseCnt), 
                  dim_in=_dim_in, 
                  dim_out=_dim_out, 
                  kernel=1, 
                  pad=0, 
                  stride=1,
                  no_bias=True)
        # add bn
        brew.spatial_bn(self.model, 
                        "pointwise%d" % (self.pointWiseCnt),
                        "pointwise%d_spatbn" % (self.pointWiseCnt),                         
                        _dim_out,
                        epsilon=1e-3, 
                        is_test=True,
                        no_bias=False
                       )
        # add relu
        brew.relu(self.model, 
                  "pointwise%d_spatbn" % (self.pointWiseCnt), 
                  "pointwise%d_relu" % (self.pointWiseCnt)
                 )
        
        self.previousBlob = "pointwise%d_relu" % (self.pointWiseCnt)
        
        self.depthWiseCnt += 1
        self.pointWiseCnt += 1
        
        return
    
    def addAvgpoolAndFcAndSoftmax(self):
        brew.average_pool(self.model, self.previousBlob, "average_pool", kernel=int(10 * self.width_mult))
        brew.conv(self.model, "average_pool", "conv_final", kernel=1, dim_in=(1024 * self.width_mult), dim_out=3, no_bias=False)
        brew.softmax(self.model, "conv_final", "softmax")

In [3]:
raw_data = np.random.randn(1, 3, 160, 160).astype(np.float32)
workspace.FeedBlob("data", raw_data)
mobilenet_model = model_helper.ModelHelper(name="mobilenet")
builder = MobileNetBuilder(mobilenet_model, width_mult=0.5)
builder.addInputDataAndStandConv("data")

builder.addDepthwiseConvAndPointWiseConv(filter_in=32, filter_out=64, isDownSample=False)
builder.addDepthwiseConvAndPointWiseConv(filter_in=64, filter_out=128, isDownSample=True)
builder.addDepthwiseConvAndPointWiseConv(filter_in=128, filter_out=128, isDownSample=False)
builder.addDepthwiseConvAndPointWiseConv(filter_in=128, filter_out=256, isDownSample=True)
builder.addDepthwiseConvAndPointWiseConv(filter_in=256, filter_out=256, isDownSample=False)
builder.addDepthwiseConvAndPointWiseConv(filter_in=256, filter_out=512, isDownSample=True)

for i in range(5):
    builder.addDepthwiseConvAndPointWiseConv(filter_in=512, filter_out=512, isDownSample=False)

builder.addDepthwiseConvAndPointWiseConv(filter_in=512, filter_out=1024, isDownSample=True)
builder.addDepthwiseConvAndPointWiseConv(filter_in=1024, filter_out=1024, isDownSample=False)
builder.addAvgpoolAndFcAndSoftmax()

In [4]:
workspace.RunNetOnce(mobilenet_model.param_init_net)
workspace.CreateNet(mobilenet_model.net)
workspace.RunNet(mobilenet_model.net)

True

In [5]:
def convert_hwcincout_to_coutcinhw(tensor_in):
    '''
    parameter in tensorflow was organized by H x W x INPUT_CHANEL x OUTPUT_CHANEL
    parameter in caffe2 was organized by OUTPUT_CHANEL x INPUT_CHANEL x H x W
    '''
    pass
    if len(tensor_in.shape) == 1:
        return tensor_in
    ans_np = np.rollaxis(np.rollaxis(tensor_in, 3), 3, start=1)
    # when we use group_conv and channel per group is 1, 
    # we need to reshape tensor to make input channel be 1
    if ans_np.shape[0] == 1:
        return np.rollaxis(ans_np, 1)
    return ans_np

graph = tf.Graph()
sess = tf.Session(graph=graph)

with graph.as_default():
    mobilenet_func = mobilenet_lib.mobilenet_factory(depth_multiplier=0.5, default_image_size=160, scope="MobilenetV1")
    mobilenet_func(tf.placeholder(tf.float32, shape=(None, 160, 160, 3)), 3, False)
    
    saver = tf.train.Saver()
    saver.restore(sess, "/world/data-c22/AR/damo_porn_weighting_4/model.ckpt")

In [6]:
# from IPython import display
# graph = net_drawer.GetPydotGraph(mobilenet_model.net.Proto().op[:50], "", rankdir="LR")
# display.Image(graph.create_png(), width=800)

In [7]:
def export(INIT_NET, PREDICT_NET, model) :

    with open(PREDICT_NET, 'wb') as f:
        model.net._net.external_output.extend(["softmax"])
        f.write(model.net._net.SerializeToString())
    init_net = caffe2_pb2.NetDef()

    # get Variable from model of tensorflow
    with graph.as_default():
        variables_from_tf_model_lst = [v for v in tf.trainable_variables() if (v.name.find("beta") == -1) ]
        blobref_from_c2_model_lst = [v for v in model.params if (str(v).endswith("spatbn_s") is False) and (str(v).find("gconv") == -1) and (str(v).find("spatbn_b") ==-1)]
        
        for i, v in enumerate(variables_from_tf_model_lst):
            blob_raw = sess.run(v)
            blob = convert_hwcincout_to_coutcinhw(blob_raw)
    
            op = core.CreateOperator("GivenTensorFill", [], [blobref_from_c2_model_lst[i]],arg=[ utils.MakeArgument("shape", blob.shape),utils.MakeArgument("values", blob)])
            init_net.op.extend([op])
            print "Converting %s of tensorflow ---> %s of caffe2" %(v.name, str(blobref_from_c2_model_lst[i]))
            print v.get_shape(), "--->", blob.shape
                
        # handle BN parameters of depthwise and pointwise convolution
        for i in range(1, 14):
            dw_tensor_bn_b = tf.get_default_graph().get_tensor_by_name("MobilenetV1/Conv2d_%d_depthwise/BatchNorm/beta:0" %(i))
            dw_tensor_bn_mm = tf.get_default_graph().get_tensor_by_name("MobilenetV1/Conv2d_%d_depthwise/BatchNorm/moving_mean:0" %(i))
            dw_tensor_bn_mv = tf.get_default_graph().get_tensor_by_name("MobilenetV1/Conv2d_%d_depthwise/BatchNorm/moving_variance:0" %(i))
            pw_tensor_bn_b = tf.get_default_graph().get_tensor_by_name("MobilenetV1/Conv2d_%d_pointwise/BatchNorm/beta:0" %(i))
            pw_tensor_bn_mm = tf.get_default_graph().get_tensor_by_name("MobilenetV1/Conv2d_%d_pointwise/BatchNorm/moving_mean:0" %(i))
            pw_tensor_bn_mv = tf.get_default_graph().get_tensor_by_name("MobilenetV1/Conv2d_%d_pointwise/BatchNorm/moving_variance:0" %(i))
            bn_params_tf_lst = sess.run([dw_tensor_bn_b, 
                                         dw_tensor_bn_mm, 
                                         dw_tensor_bn_mv, 
                                         pw_tensor_bn_b, 
                                         pw_tensor_bn_mm, 
                                         pw_tensor_bn_mv
                                        ])

            op = core.CreateOperator("GivenTensorFill", [], ["depthwise%d_spatbn_b" %(i)],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[0].shape),utils.MakeArgument("values", bn_params_tf_lst[0])])
            init_net.op.extend([op])
            op = core.CreateOperator("GivenTensorFill", [], ["depthwise%d_spatbn_rm" %(i)],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[1].shape),utils.MakeArgument("values", bn_params_tf_lst[1])])
            init_net.op.extend([op])
            op = core.CreateOperator("GivenTensorFill", [], ["depthwise%d_spatbn_riv" %(i)],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[2].shape),utils.MakeArgument("values", bn_params_tf_lst[2])])
            init_net.op.extend([op])
            op = core.CreateOperator("GivenTensorFill", [], ["depthwise%d_spatbn_s" %(i)],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[0].shape),utils.MakeArgument("values", np.ones(bn_params_tf_lst[0].shape))])
            init_net.op.extend([op])
            op = core.CreateOperator("GivenTensorFill", [], ["pointwise%d_spatbn_b" %(i)],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[3].shape),utils.MakeArgument("values", bn_params_tf_lst[3])])
            init_net.op.extend([op])
            op = core.CreateOperator("GivenTensorFill", [], ["pointwise%d_spatbn_rm" %(i)],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[4].shape),utils.MakeArgument("values", bn_params_tf_lst[4])])
            init_net.op.extend([op])
            op = core.CreateOperator("GivenTensorFill", [], ["pointwise%d_spatbn_riv" %(i)],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[5].shape),utils.MakeArgument("values", bn_params_tf_lst[5])])
            init_net.op.extend([op])
            op = core.CreateOperator("GivenTensorFill", [], ["pointwise%d_spatbn_s" %(i)],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[3].shape),utils.MakeArgument("values", np.ones(bn_params_tf_lst[3].shape))])
            init_net.op.extend([op])
            
        # handle BN parameters of full convolution
        tensor_bn_b = tf.get_default_graph().get_tensor_by_name("MobilenetV1/Conv2d_0/BatchNorm/beta:0")
        tensor_bn_mm = tf.get_default_graph().get_tensor_by_name("MobilenetV1/Conv2d_0/BatchNorm/moving_mean:0")
        tensor_bn_mv = tf.get_default_graph().get_tensor_by_name("MobilenetV1/Conv2d_0/BatchNorm/moving_variance:0")
        bn_params_tf_lst = sess.run([tensor_bn_b, tensor_bn_mm, tensor_bn_mv])

        op = core.CreateOperator("GivenTensorFill", [], ["conv1_spatbn_b"],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[0].shape),utils.MakeArgument("values", bn_params_tf_lst[0])])
        init_net.op.extend([op])
        op = core.CreateOperator("GivenTensorFill", [], ["conv1_spatbn_rm"],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[1].shape),utils.MakeArgument("values", bn_params_tf_lst[1])])
        init_net.op.extend([op])
        op = core.CreateOperator("GivenTensorFill", [], ["conv1_spatbn_riv"],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[2].shape),utils.MakeArgument("values", bn_params_tf_lst[2])])
        init_net.op.extend([op])
        op = core.CreateOperator("GivenTensorFill", [], ["conv1_spatbn_s"],arg=[ utils.MakeArgument("shape", bn_params_tf_lst[0].shape),utils.MakeArgument("values", np.ones(bn_params_tf_lst[0].shape))])
        init_net.op.extend([op])
                
    init_net.op.extend([core.CreateOperator("ConstantFill", [], ["data"], shape=(1, 3, 160,160))])

    with open(INIT_NET, 'wb') as f:
        f.write(init_net.SerializeToString())
        
export("convert_mobilenet_model_from_tensorflow_to_caffe2_init_net.pb", 
       "convert_mobilenet_model_from_tensorflow_to_caffe2_predict_net.pb",
       mobilenet_model)

Converting MobilenetV1/Conv2d_0/weights:0 of tensorflow ---> conv1_w of caffe2
(3, 3, 3, 16) ---> (16, 3, 3, 3)
Converting MobilenetV1/Conv2d_1_depthwise/depthwise_weights:0 of tensorflow ---> depthwise1_w of caffe2
(3, 3, 16, 1) ---> (16, 1, 3, 3)
Converting MobilenetV1/Conv2d_1_pointwise/weights:0 of tensorflow ---> pointwise1_w of caffe2
(1, 1, 16, 32) ---> (32, 16, 1, 1)
Converting MobilenetV1/Conv2d_2_depthwise/depthwise_weights:0 of tensorflow ---> depthwise2_w of caffe2
(3, 3, 32, 1) ---> (32, 1, 3, 3)
Converting MobilenetV1/Conv2d_2_pointwise/weights:0 of tensorflow ---> pointwise2_w of caffe2
(1, 1, 32, 64) ---> (64, 32, 1, 1)
Converting MobilenetV1/Conv2d_3_depthwise/depthwise_weights:0 of tensorflow ---> depthwise3_w of caffe2
(3, 3, 64, 1) ---> (64, 1, 3, 3)
Converting MobilenetV1/Conv2d_3_pointwise/weights:0 of tensorflow ---> pointwise3_w of caffe2
(1, 1, 64, 64) ---> (64, 64, 1, 1)
Converting MobilenetV1/Conv2d_4_depthwise/depthwise_weights:0 of tensorflow ---> depthwise

In [8]:
# print mobilenet_model.net.Proto()